In [6]:
# packages
import numpy as np
import pandas as pd
import random
import os

# database
import yfinance as yf
from sqlalchemy import create_engine, inspect

# quadratic
import quadprog
from scipy.optimize import minimize

# visualisation
import seaborn as sns
import itertools
import matplotlib.pyplot as plt
from tabulate import tabulate


# maybe
import plotly.express as px
import matplotlib
from matplotlib.patches import Patch
plt.rcParams.update({'figure.max_open_warning': 0})
plt.style.use('fivethirtyeight')
cmap_data = plt.cm.Paired
cmap_cv = plt.cm.coolwarm

/Users/jackstevenson/CQF/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [17]:
#### Data & Pre-Processing

In [18]:
# create our engine
engine = create_engine("sqlite:///project_portfolio.db")

df = pd.read_sql('portfolio_data', con=engine)

# # set date as index
df.set_index('Date', inplace=True)

# create separate t bill dataframe
three_month_tbill = df['3M TB'] / 252

# create separate benchmark dataframe
benchmark_spx_500 = df['Benchmark - S&P 500'].pct_change()

# consistent ordering
df = df[['Energy', 'Materials', 'Industrials', 'Consumer Discretionary', 'Consumer Staples', 'Health Care', 'Financials', 'Information Technology', 
         'Communication Services', 'Utilities', 'Real Estate', 'Volatility (Exo)', 'Commodities (Exo)', 'Bonds (Exo)', 'Factor SMB', 'Factor HML', 
         'Factor RMW', 'Factor CMA']]

# calculate simple returns
simple_returns = df.drop(columns=['Factor SMB', 'Factor HML', 'Factor RMW', 'Factor CMA']).pct_change()

simple_returns = pd.concat([simple_returns, df[['Factor SMB', 'Factor HML', 'Factor RMW', 'Factor CMA']]], axis=1)

/var/folders/3z/lz8knlfn5ys693qpn6bnjb440000gn/T/ipykernel_36581/3435988170.py:21: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  simple_returns = df.drop(columns=['Factor SMB', 'Factor HML', 'Factor RMW', 'Factor CMA']).pct_change()


#### Custom Factor - Momentum Factor

In [19]:
# lookback period 252 (approximate 12 months)
lookback_period = 252

# exclude the exogenous and fama-french
sector_momentum = simple_returns.drop(columns=['Volatility (Exo)', 'Commodities (Exo)', 
                                               'Bonds (Exo)','Factor SMB', 'Factor HML', 
                                               'Factor RMW', 'Factor CMA']).rolling(window=lookback_period).apply(lambda x: (x + 1).prod() - 1)

# rank the sector returns over lookback plus lag to prevent perfect portfolio
momentum_ranks = sector_momentum.rank(axis=1, method='first', ascending=False).shift(1).dropna()

# identify top and bottom
top_momentum_sectors = momentum_ranks <= 3  # Top 3 sectors
bottom_momentum_sectors = momentum_ranks >= 9  # Bottom 3 sectors

# construct portfolio returns from top & bottom
momentum_high_portfolio = simple_returns[top_momentum_sectors].mean(axis=1)
momentum_low_portfolio = simple_returns[bottom_momentum_sectors].mean(axis=1)

# calculate momentum factor returns
momentum_factor_return = momentum_high_portfolio - momentum_low_portfolio
momentum_factor_return_df = momentum_factor_return.to_frame(name='Factor MOM')

# add into simple_returns_df
simple_returns = pd.concat([simple_returns, momentum_factor_return_df], axis=1)

# drop the first 12 months due to lookback
simple_returns.dropna(inplace=True)

In [20]:
# subtract the 3m t bill daily rate
excess_returns = simple_returns.sub(three_month_tbill, axis=0)

# calculate log returns returns
log_returns = np.log(1 + simple_returns).dropna()

#### Black-Litterman Class Model

In [21]:
class BlackLittermanModel:
    def __init__(self, equilibrium_weights, log_returns, risk_aversion, views_mapping_matrix, views_matrix, tracking_error_target=0.00000009, tau=0.025):
        self._equilibrium_weights = equilibrium_weights
        self._log_returns = log_returns
        self._risk_aversion = risk_aversion
        self._tau = tau
        self._covariance_matrix = self._calculate_covariance_matrix()
        self._views_mapping_matrix = views_mapping_matrix
        self._views_matrix = views_matrix
        self._tracking_error_target = tracking_error_target 

    # TODO: private methods
    def _calculate_covariance_matrix(self):
        # calculate the vols
        volatilities_array = self._log_returns.std()

        # calculate correlation
        correlation_coefficients = self._log_returns.corr()

        # create the diagonal vol matrix (vol on the diagonal, zeros elsewhere)
        std_diag_vol_matrix  = np.diag(volatilities_array)

        # compute the covariance matrix
        covariance_matrix = std_diag_vol_matrix @ correlation_coefficients.values @ std_diag_vol_matrix

        # pass the headers from log return 

        covariance_matrix = pd.DataFrame(
            covariance_matrix,
            columns = self._log_returns.columns,
            index = self._log_returns.columns,
        )

        return covariance_matrix
    
    def calculate_views_adjusted_returns(self):
        # we need implied equity returns and omega
        implied_returns_vector = self.calculate_implied_equilibrium_returns().values

        omega = self.calculate_uncertainty_views_matrix()

        # we have three terms for the np.dot product
        tau_sigma_transpose_p = self._tau * (self._covariance_matrix.values @ self._views_mapping_matrix.T)

        inverse_middle_term = np.linalg.inv((self._views_mapping_matrix @ tau_sigma_transpose_p) + omega)

        view_minus_p = (self._views_matrix - (self._views_mapping_matrix @ implied_returns_vector))

        views_adjusted_returns_vector = implied_returns_vector + ((tau_sigma_transpose_p @ inverse_middle_term) @ view_minus_p)

        # convert to dataframe for consistency
        views_adjusted_returns = pd.DataFrame(views_adjusted_returns_vector, columns=["Adjusted Return"], index=self._log_returns.columns)

        return views_adjusted_returns
    
    # calculate the covariance of the error terms aka omega
    def calculate_uncertainty_views_matrix(self):
        # you extract the diagonal and then convert to a diagonal matrix with zeros
        omega = np.diag(((self._views_mapping_matrix @ (self._tau * self._covariance_matrix)) @ self._views_mapping_matrix.T))
        
        return np.diag(omega)

    
    def calculate_implied_equilibrium_returns(self):
        implied_returns_vector = self._risk_aversion * (self._covariance_matrix @ self._equilibrium_weights)

        implied_returns = pd.DataFrame(implied_returns_vector, columns=["Implied Return"], index=self._log_returns.columns)

        return implied_returns
    
    def calculate_unconstrained_mv_optimisation(self):
        # invert the covariance matrix
        inv_cov_matrix = np.linalg.inv(self._covariance_matrix)

        # calculate adjusted returns
        adjusted_returns = self.calculate_views_adjusted_returns().values

        # obtain weights
        optimal_weights = self._risk_aversion * np.dot(inv_cov_matrix, adjusted_returns)

        # # normalise the sum of weights to 1
        optimal_weights /= np.sum(optimal_weights)

        # convert to dataframe for consistency
        optimal_weights = pd.DataFrame(optimal_weights, columns=["Adjusted Weights"], index=self._log_returns.columns)

        return optimal_weights
    
    def generate_mv_constraint_matrices(self, implied_returns_vector_values):
        # we will generate our two constraints - no short selling & no leverage
        number_of_assets = len(implied_returns_vector_values)

        # no short selling
        no_short_selling_constraint_coefficient = np.eye(number_of_assets)
        no_short_selling_constraint_rhs = np.zeros(number_of_assets)

        # no leverage - you need to transpose the 
        no_leverage_constraint_coefficient = np.ones([1, number_of_assets])
        no_leverage_constraint_rhs = np.array([1.0])

        # stack them together
        C_constraint_matrix = np.vstack([no_leverage_constraint_coefficient, no_short_selling_constraint_coefficient])
        b_constraint_matrix = np.hstack([no_leverage_constraint_rhs, no_short_selling_constraint_rhs])

        return C_constraint_matrix, b_constraint_matrix
    
        
    def calculate_constrained_mv_optimisation(self):
        implied_returns_vector_values = self.calculate_implied_equilibrium_returns().values

        # we are solving an equation of the form - 1/2 x^T G x - a^T x s.t. C.T x >= b
        quadratic_G = self.risk_aversion * self._covariance_matrix.values

        quadratic_a = self.calculate_views_adjusted_returns().values.flatten()

        # generate constraints
        C_constraint_matrix, b_constraint_matrix = self.generate_mv_constraint_matrices(implied_returns_vector_values)

        # solve for weights
        optimal_weights = quadprog.solve_qp(quadratic_G, quadratic_a, C_constraint_matrix.T, b_constraint_matrix, meq=1)[0]

        optimal_weights = pd.DataFrame(optimal_weights, columns=["Constrained Adjusted Weights"], index=self._log_returns.columns)

        return optimal_weights
    
    
    def calculate_tracking_error_optimisation(self):
        implied_returns_vector_values = self.calculate_implied_equilibrium_returns().values

        number_of_assets = len(implied_returns_vector_values)

        adjusted_returns = self.calculate_views_adjusted_returns().values

        # constraints
        constraints = [{'type': 'eq', 'fun': self.zero_sum_equality_constraint},
                       {'type': 'eq', 'fun': self.tracking_error_target_constraint}]
        

        # bounds - no short selling or other constraints
        bounds = [(None, None)] * number_of_assets

        # initial guess - very small non-zero deviation
        x0 = np.ones(number_of_assets) * 1e-3

        # generate weights
        results = minimize(fun= self.objective_function, x0=x0, bounds=bounds, constraints=constraints)
        
        if not results.success:
            print("Optimization failed:", results.message)
            return None

        optimal_deviations = results.x

        # create a series from the deviations with the same index as the eq weights
        optimal_deviations = pd.Series(results.x, index=self._equilibrium_weights.index)

        optimal_weights = self._equilibrium_weights + optimal_deviations

        optimal_weights = pd.DataFrame(optimal_weights, columns=["TE Constrained Adjusted Weights"], index=self._log_returns.columns)

        return optimal_weights
    
    # Tracking Error - objective and constraints
    def objective_function(self, x):
        # maximise returns
        adjusted_returns = self.calculate_views_adjusted_returns().values
        value = - np.dot(x, adjusted_returns)
        # print(f"Objective function value: {value} for x: {x}")


        return value
    
    def zero_sum_equality_constraint(self, x):
        # constraint - sum of deviations is zero
        value = np.sum(x)
        # print(f"Zero sum constraint value: {value} for x: {x}")

        return value
    
    def tracking_error_target_constraint(self, x):
        value = np.dot(x.T, np.dot(self.covariance_matrix, x)) - self._tracking_error_target
        # print(f"Tracking error constraint value: {value} for x: {x}")

        return value


    # attributes
    # tau
    @property
    def tau(self):
        return self._tau

    @tau.setter
    def tau(self, value):
        self._tau = value

    # risk aversion 
    @property
    def risk_aversion(self):
        return self._risk_aversion

    @risk_aversion.setter
    def risk_aversion(self, value):
        self._risk_aversion = value

    @property
    def covariance_matrix(self):
        return self._covariance_matrix

In [22]:
# Initialise the class

In [24]:
# testing the class
equilibrium_weights = pd.Series({
    'Energy': 0.037,
    'Materials': 0.026,
    'Industrials': 0.08,
    'Consumer Discretionary': 0.118,
    'Consumer Staples': 0.062,
    'Health Care': 0.1330,
    'Financials': 0.1150,
    'Information Technology': 0.281,
    'Communication Services': 0.096,
    'Utilities': 0.026,
    'Real Estate': 0.026
})

risk_aversion_dict = pd.Series({
    # in order of increasing aversion
    'Kelly': 0.01,
    'Market': 2.24,
    'Trustee': 6
})

# view adjusted returns with defined views
k = 3
n = 11

#  these are annual views so we need to divide by 252 to make them daily
Q = np.array([0.07, 0.0025, 0.01]).reshape(-1, 1) / 252

P = np.zeros((k, n))

# bl_model.calculate_views_adjusted_returns()
# first view
P[0, 0] = 1
# second view
P[1, 6] = -1
P[1, 7] = 1
# third view
P[2, 3] = -0.24
P[2, 4] = 0.71
P[2, 7] = -0.76
P[2, 9] = 0.29

# ------------------------------------ WRITE UP ------------------------------------

#### 1.4 What is the Black-Litterman Portfolio

The initial formulation of the Black-Litterman model for Portfolio construction was described in the 1992 paper. The paper described an intuitive solution by combining the mean-variance opitimisation framework of Markowitz and the capital asset pricing model (CAPM) of Sharpe and Lintner. The issue with the CAPM model and minimum-variance portfolio decribed by Markowitz is that small tweaks in the values could lead to significant rebalancing of the portfolio. The Markowitz framework effectively takes the risk-free return as the alternative benchmark. The Black-Litterman model on the otherhand uses equilibrium risk premiums to provide a neutral reference point for expected returns, generating a market-capitalisation-weighted-portfolio and then incorporating subjective views that tilts in the direction of assets favored by the investor to enable the generation of alpha. The Black-Litterman models assumes does not assume the model is always at CAPM equilibrium, but any shift away from this equilibrium will experience pressure from the market to revert. Furthermore, the model allows the investor to have as many, or as few, views as they wish on either an absolute or relative basis. 

We will undertake the following steps in the next few sections to construct our portfolio according to the Black-Litterman framework. Firstly, we need to assign our portfolio weights based upon the market capitalisation of our benchmark portfolio. As previously outlined we will assign 92% of our portfolio to the S&P 500 sectors that we have proxied via iShares ETFs and will according allocate based on their weight in the S&P 500 in 2022. The remaining 8% of the portfolio will be equally split across our three exogenous factors, four Fama-French factors and custom momentum factor. Based on these weights we will derive the implied market returns for each of our factors. Due to this method of deriving the market implied returns the frame work does not recommend deviating from the market returns until you formulate views about the factor that cause them to deviate on either an absolute or relative basis from the implied returns. Once we have formulated our views on the market then we will combine our market implied returns and views via Bayes Theorem to construct a posterior returns distribution. 

We will then use our posterior returns distribution to calculate our views adjusted optimal weights for our portfolio. Finally, once we have completed this optimisation we will compare how our new portfolio performs vs the benchmark S&P from 2022 to 2024. 

While the Markowitz mean-variance optimisation is designed to use discrete returns, the Black-Litterman model is based on log return which we will use here. 

$$

R_{i} = log(1 + \frac{P_{i+1}}{P_i})

$$

Fig. 6 Default Weights from S&P 500 per GICs Sector (2022)

| Sector                  | Weights  |
| :---------------------: | :---:    |
| Information Technology  | 28.10%   |
| Financials              | 11.50%   |
| Health Care             | 13.30%   |
| Consumer Discretionary  | 11.8%    |
| Communication           | 9.60%    |
| Industrials             | 8.00%    |
| Consumer Staples        | 6.20%    |
| Energy                  | 3.70%    |
| Utilities               | 2.60%    |
| Real Estate             | 2.60%    |
| Materials               | 2.60%    |

In [ ]:
1.5 Prior 

#### 2.2 Reverse Optimisation & Prior Returns

Now that we have the optimal weights according to our equilibrium returns we are going to undertake a reverse optimisation of the weights to arrive at the returns vector for our sectors that would generate these weights. To solve this reverse optimisation we need to set up our problem. 

$$
arg \min_{\omega} \omega'\pi - \lambda\omega'\Sigma\omega
$$

- $\omega$: a vector of our sector weights
- $\pi$: a vector of our equilibrium returns
- $\lambda$: a scalar factor for risk-aversion
- $\Sigma$: covariance matrix for our returns

Initially we will solve an unconstrained mean-variance and then will introduce constraints for a more robust solution. We take the derivative with respect to $w$ to get the following equation: 

$$
\pi - 2 \lambda \Sigma w = 0
$$

We re-arrange the equation for both $\pi$ and $w$ to get the implied equilibrium return from our market weights and optimal weights under our model:

$$
\pi^{*} = 2 \lambda \Sigma w
$$

$$
w^{*} = \frac{1}{2 \lambda} \Sigma^{-1} \pi
$$

We can now calculate the implied market returns from our model based upon the market weights provided by the corresponding ETF proxies.